In [1]:
import os
import re
import requests
from bs4 import BeautifulSoup

In [2]:
accepted_base_url = 'https://steamcommunity.com/workshop/browse/?appid=440&browsesort=accepted&section=mtxitems&browsefilter=accepted'
not_accepted_base_url = 'https://steamcommunity.com/workshop/browse/?appid=440&browsesort=trend&section=mtxitems'


In [3]:
def get_soup(url: str):
    return BeautifulSoup(requests.get(url).content, 'html.parser')


In [4]:
def download_item_by_id(folder_name: str, id: str):
    with open(f'{folder_name}{os.sep}{id}.html', "wb") as item_file:
        item_file.write(requests.get(
            f"https://steamcommunity.com/sharedfiles/filedetails/?id={id}").content)


In [5]:
def download_items(folder_name: str, base_url: str):
    os.makedirs(folder_name, exist_ok=True)

    base_soup = get_soup(base_url)

    page_count = int(base_soup.select('.pagelink')[-1].text.replace(',', ''))

    paging_info = base_soup.select_one('.workshopBrowsePagingInfo').text
    item_count = int(
        re.search(r'of ([\d,]+) entries', paging_info).group(1).replace(',', ''))

    print(page_count, 'pages', item_count, 'items')

    item_count = 1

    for page_number in range(1, page_count+1):
        print('started page', page_number)

        page_soup = get_soup(base_url+f'&p={page_number}')
        page_item_ids = [e.find('a').attrs['data-publishedfileid']
                         for e in page_soup.select('.workshopItem')]

        for id in page_item_ids:
            if os.path.exists(f'{folder_name}/{id}.html'):
                continue

            print('downloading item number', item_count, 'with id:', id)
            download_item_by_id(folder_name,id)

            item_count = item_count + 1


In [6]:
download_items('accepted',accepted_base_url)

62 pages 1849 items
started page 1
started page 2
started page 3
started page 4
started page 5
started page 6
started page 7
started page 8
started page 9
started page 10
started page 11
started page 12
started page 13
started page 14
started page 15
started page 16
started page 17
started page 18
started page 19
started page 20
started page 21
started page 22
started page 23
started page 24
started page 25
started page 26
started page 27
started page 28
started page 29
started page 30
started page 31
started page 32
started page 33
started page 34
started page 35
started page 36
started page 37
started page 38
started page 39
started page 40
started page 41
started page 42
started page 43
started page 44
started page 45
started page 46
started page 47
started page 48
started page 49
started page 50
started page 51
started page 52
started page 53
started page 54
started page 55
started page 56
started page 57
started page 58
started page 59
started page 60
started page 61
started page 

In [7]:
download_items('not_accepted',not_accepted_base_url)

705 pages 21144 items
started page 1
downloading item number 1 with id: 2826724523
downloading item number 2 with id: 2826806091
downloading item number 3 with id: 2826725310
downloading item number 4 with id: 2826725070
downloading item number 5 with id: 2826724776
downloading item number 6 with id: 2826526224
downloading item number 7 with id: 2826806620
downloading item number 8 with id: 2826557451
downloading item number 9 with id: 2826689723
downloading item number 10 with id: 2826558085
downloading item number 11 with id: 2826551556
downloading item number 12 with id: 2826553235
downloading item number 13 with id: 2826528270
downloading item number 14 with id: 2824614449
downloading item number 15 with id: 2826231160
downloading item number 16 with id: 2826230414
downloading item number 17 with id: 2826452302
downloading item number 18 with id: 2826050652
downloading item number 19 with id: 2825995178
downloading item number 20 with id: 2826049104
downloading item number 21 with 